In [1]:
import pandas as pd
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from copy import deepcopy
import json
from sklearn.preprocessing import StandardScaler

c:\Users\Konrad\miniconda3\envs\uma\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
df = pd.read_csv(path + '/creditcard.csv')

In [3]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [4]:
df = df.drop(columns="Time")
scaler = StandardScaler()
for col in df.columns:
    if col == "Class":
        continue
    df[col] = scaler.fit_transform(df[[col]])

In [5]:
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000
mean,-1.506872e-17,-9.580116e-18,-8.622104e-17,-5.189230e-18,3.832046e-17,9.979288e-18,1.237432e-17,-3.193372e-18,7.234983e-19,5.588401e-18,...,-3.642440e-18,3.193372e-18,8.781773e-18,9.580116e-18,-5.029561e-17,5.887780e-18,2.444925e-18,-7.908585e-18,2.913952e-17,0.001727
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,0.041527
min,-2.879855e+01,-4.403529e+01,-3.187173e+01,-4.013919e+00,-8.240810e+01,-1.963606e+01,-3.520940e+01,-6.130252e+01,-1.222802e+01,-2.258191e+01,...,-4.741907e+01,-1.506565e+01,-7.175446e+01,-4.683638e+00,-1.975033e+01,-5.401098e+00,-5.590660e+01,-4.674612e+01,-3.532294e-01,0.000000
25%,-4.698918e-01,-3.624707e-01,-5.872142e-01,-5.993788e-01,-5.010686e-01,-5.766822e-01,-4.478860e-01,-1.746805e-01,-5.853631e-01,-4.917360e-01,...,-3.109433e-01,-7.473476e-01,-2.591784e-01,-5.854676e-01,-6.084001e-01,-6.780717e-01,-1.755053e-01,-1.604440e-01,-3.308401e-01,0.000000
50%,9.245351e-03,3.965683e-02,1.186124e-01,-1.401724e-02,-3.936682e-02,-2.058046e-01,3.241723e-02,1.871982e-02,-4.681169e-02,-8.533551e-02,...,-4.009429e-02,9.345377e-03,-1.792420e-02,6.765678e-02,3.183240e-02,-1.081217e-01,3.325174e-03,3.406368e-02,-2.652715e-01,0.000000
75%,6.716939e-01,4.867202e-01,6.774569e-01,5.250082e-01,4.433465e-01,2.991625e-01,4.611107e-01,2.740785e-01,5.435305e-01,4.168842e-01,...,2.537392e-01,7.283360e-01,2.364319e-01,7.257153e-01,6.728006e-01,4.996663e-01,2.255648e-01,2.371526e-01,-4.471707e-02,0.000000
max,1.253351e+00,1.335775e+01,6.187993e+00,1.191874e+01,2.521413e+01,5.502015e+01,9.747824e+01,1.675153e+01,1.419494e+01,2.180758e+01,...,3.703471e+01,1.447304e+01,3.607668e+01,7.569684e+00,1.442532e+01,7.293975e+00,7.831940e+01,1.025434e+02,1.023622e+02,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 30 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284807 non-null  float64
 1   V2      284807 non-null  float64
 2   V3      284807 non-null  float64
 3   V4      284807 non-null  float64
 4   V5      284807 non-null  float64
 5   V6      284807 non-null  float64
 6   V7      284807 non-null  float64
 7   V8      284807 non-null  float64
 8   V9      284807 non-null  float64
 9   V10     284807 non-null  float64
 10  V11     284807 non-null  float64
 11  V12     284807 non-null  float64
 12  V13     284807 non-null  float64
 13  V14     284807 non-null  float64
 14  V15     284807 non-null  float64
 15  V16     284807 non-null  float64
 16  V17     284807 non-null  float64
 17  V18     284807 non-null  float64
 18  V19     284807 non-null  float64
 19  V20     284807 non-null  float64
 20  V21     284807 non-null  float64
 21  V22     28

In [7]:
X, y = df.drop(columns="Class"), df["Class"]

### Undersampling

In [8]:
class_counts = y.value_counts()
minority_class = class_counts.idxmin()
minority_count = class_counts.min()
majority_class = class_counts.idxmax()

desired_ratio = 256
majority_count = minority_count * desired_ratio
sampling_strategy = {
    minority_class: minority_count,
    majority_class: majority_count
    }

In [9]:
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=17)
X_resampled, y_resampled = rus.fit_resample(X, y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=17)

In [11]:

print(f"Imbalance ratio in training set: {(y_train.count()-np.sum(y_train))/np.sum(y_train)}")
print(f"Class proportion in testing set: {(y_test.count()-np.sum(y_test))/np.sum(y_test)}")

Imbalance ratio in training set: 261.7402597402597
Class proportion in testing set: 235.34579439252337


Creating new smaller training data set that has both classes:

In [12]:
rng = np.random.RandomState(42)
labeled_X = pd.DataFrame()
labeled_y = pd.Series()
chosen_indexes = []
chosen_0_class = 0
chosen_1_class = 0
for i in y_train.index.tolist():
    class_value = y_train.loc[i]
    if class_value == 0 and chosen_0_class < desired_ratio:
        chosen_0_class += 1
    elif class_value == 1 and chosen_1_class < 1:
        chosen_1_class += 1
    elif chosen_0_class == desired_ratio and chosen_1_class == 1:
        break
    else:
        continue
    labeled_X = pd.concat([labeled_X, X_train.loc[[i]]])
    X_train = X_train.drop(i)
    labeled_y = pd.concat([labeled_y, y_train.loc[[i]]])
    y_train = y_train.drop(i)
    
    

# while not (labeled_y_sum > 0 and len(labeled_y) != labeled_y_sum):
#     random_index = rng.choice(y_train.index.tolist())
#     labeled_y = pd.concat([labeled_y, y_train.loc[[random_index]]])
#     y_train = y_train.drop(random_index)
#     labeled_X = pd.concat([labeled_X, X_train.loc[[random_index]]])
#     X_train = X_train.drop(random_index)
#     labeled_y_sum = np.sum(labeled_y)


C:\Users\Konrad\AppData\Local\Temp\ipykernel_5256\1620467898.py:19: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  labeled_y = pd.concat([labeled_y, y_train.loc[[i]]])


In [13]:
labeled_y

167573    0
149330    0
56751     0
210484    0
11358     0
         ..
47216     0
129654    0
272357    0
122345    0
112488    0
Length: 257, dtype: int64

In [14]:
def get_one_class_probability(binary_proba):
    single_values = []
    for proba in binary_proba:
        single_values.append(round(proba[1], 4))
    return single_values

In [15]:
labeled_X_active = deepcopy(labeled_X)
labeled_X_passive = deepcopy(labeled_X)
labeled_y_active = deepcopy(labeled_y)
labeled_y_passive = deepcopy(labeled_y)

unlabeled_X_active = deepcopy(X_train)
unlabeled_X_passive = deepcopy(X_train)
unlabeled_y_active = deepcopy(y_train)
unlabeled_y_passive = deepcopy(y_train)


Active Learning


In [ ]:
ITERS = 300
results = {
    "training set size": len(X_train),
    "count of minority class members": int(np.sum(y_train)),
    "true labels": [int(i) for i in y_test.values],
    "models": []}
active_model = SVC(kernel="rbf", probability=True, cache_size=1000)
for i in range(ITERS):
    active_model.fit(labeled_X_active, labeled_y_active)
    
    test_proba = active_model.predict_proba(X_test)
    one_class_proba = get_one_class_probability(test_proba)
    model = {"training instances": len(labeled_y_active), 
             "predictions": one_class_proba,
             "minority_class_examples_used": sum(labeled_y_active)}
    results["models"].append(model)
    
    probabilities = active_model.predict_proba(unlabeled_X_active)
    uncertainty = np.abs(probabilities[:, 0] - probabilities[:, 1])
    least_confident_index = np.argmin(uncertainty)
    least_confident_sample = unlabeled_X_active.iloc[least_confident_index]
    df_index_number = least_confident_sample.name
    
    labeled_X_active = pd.concat([labeled_X_active, least_confident_sample.to_frame().T])
    labeled_y_active = pd.concat([labeled_y_active, unlabeled_y_active.loc[[df_index_number]]])
    unlabeled_X_active = unlabeled_X_active.drop(df_index_number)
    unlabeled_y_active = unlabeled_y_active.drop(df_index_number)
    print(f"{i}/{ITERS-1}")



0/299
1/299
2/299
3/299
4/299
5/299
6/299


In [126]:
with open(f"one_positive_class/rbf_active_learning_IR{desired_ratio}.json", "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4, separators=(',', ': '))

In [127]:
ITERS = 300
results = {
    "training set size": len(X_train),
    "count of minority class members": int(np.sum(y_train)),
    "true labels": [int(i) for i in y_test.values],
    "models": []}
active_model = SVC(kernel="rbf", probability=True, cache_size=1000)
for i in range(ITERS):
    active_model.fit(labeled_X_passive, labeled_y_passive)
    
    test_proba = active_model.predict_proba(X_test)
    one_class_proba = get_one_class_probability(test_proba)
    model = {"training instances": len(labeled_y_passive), 
             "predictions": one_class_proba,
             "minority_class_examples_used": sum(labeled_y_passive)}
    results["models"].append(model)
    
    random_record = unlabeled_X_passive.sample(n=1)
    labeled_X_passive = pd.concat([labeled_X_passive, random_record])
    labeled_y_passive = pd.concat([labeled_y_passive, unlabeled_y_passive.loc[random_record.index]])
    
    unlabeled_X_passive = unlabeled_X_passive.drop(random_record.index)
    unlabeled_y_passive = unlabeled_y_passive.drop(random_record.index)
    
    print(f"{i}/{ITERS-1}")


0/499
1/499
2/499
3/499
4/499
5/499
6/499
7/499
8/499
9/499
10/499
11/499
12/499
13/499
14/499
15/499
16/499
17/499
18/499
19/499
20/499
21/499
22/499
23/499
24/499
25/499
26/499
27/499
28/499
29/499
30/499
31/499
32/499
33/499
34/499
35/499
36/499
37/499
38/499
39/499
40/499
41/499
42/499
43/499
44/499
45/499
46/499
47/499
48/499
49/499
50/499
51/499
52/499
53/499
54/499
55/499
56/499
57/499
58/499
59/499
60/499
61/499
62/499
63/499
64/499
65/499
66/499
67/499
68/499
69/499
70/499
71/499
72/499
73/499
74/499
75/499
76/499
77/499
78/499
79/499
80/499
81/499
82/499
83/499
84/499
85/499
86/499
87/499
88/499
89/499
90/499
91/499
92/499
93/499
94/499
95/499
96/499
97/499
98/499
99/499
100/499
101/499
102/499
103/499
104/499
105/499
106/499
107/499
108/499
109/499
110/499
111/499
112/499
113/499
114/499
115/499
116/499
117/499
118/499
119/499
120/499
121/499
122/499
123/499
124/499
125/499
126/499
127/499
128/499
129/499
130/499
131/499
132/499
133/499
134/499
135/499
136/499
137/499
138/49

In [128]:
with open(f"one_positive_class/rbf_passive_learning_IR{desired_ratio}.json", "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4, separators=(',', ': '))